In [1]:
# !pip install pymysql
import sys

import numpy as np
import pandas as pd
import pymysql as ps

sys.setrecursionlimit(100000)
conn = ps.connect(host="localhost", 
                  user="root", password="1234wasd",
                  db='Henry', 
                  autocommit=True)

DIR = "CSV/"
cursor = conn.cursor()


def exequery(q: str, tp=None):
    cursor.execute(q, tp)
    # conn.commit()


exequery("use M3_2")


def csvread(ar: str, delim: str):
    path = f"{DIR}{ar}"
    archivo = pd.read_csv(path, index_col=False, delimiter=delim)
    archivo.fillna("", inplace=True)

    return archivo

# __CREACIÓN DE TABLAS__

## CANAL VENTA

In [2]:
q = "drop table if exists Canal_Venta;"
exequery(q)
q = """CREATE TABLE Canal_Venta (
            Codigo int,
            Descricpion varchar(50));"""
exequery(q)

## TIPO DE GASTO

*Eliminar*

In [3]:
q = "drop table if exists Tipo_Gasto;"
exequery(q)

*CREAR*

In [4]:
q = "create table Tipo_Gasto ( IDTipoGasto int(11) not null, Descripcion varchar(100) not null, Monto_Aproximado decimal(10,2) not null);"
exequery(q)

## CLIENTE

In [5]:
q = "drop table if exists Cliente;"
exequery(q)
q = '''create table if not exists Cliente (
    IDCliente                   int not null,
    Nombre_Y_Apellido           varchar(50) not null,
    Provincia                   varchar(125),
    Domicilio                   varchar(150),
    Telefono                    varchar(50),
    Edades                      int,
    Localidad                   varchar(150),
    X                           varchar(100),
    Y                           varchar(100),
    col10                       varchar(1)
    );'''
exequery(q)

_CREAR_

## PROVEEDOR

_ELIMINAR_

In [6]:
q = "drop table if exists Proveedor;"
exequery(q)

In [7]:
q = """create table if not exists Proveedor(
    IDProveedor  int not null,
    Nombre       varchar(25) not null,
    Dirreccion   varchar(150),
    Ciudad       varchar(125),
    Región       varchar(125),
    País         varchar(125),
    Departamento varchar(125));"""
exequery(q)

## PRODUCTO

In [8]:
q = "drop table if exists Producto;"
exequery(q)

q = """create table if not exists Producto (
    IDProducto          int not null,
    Concepto            varchar(250),
    Tipo                varchar(50),
    Precio              decimal(15,2) not null
    );"""
exequery(q)

## EMPLEADO

In [9]:
q = "drop table if exists Empleado;"
exequery(q)
q = '''create table if not exists Empleado (
    IDEmpleado   int not null,
    Apellido     varchar(25) not null,
    Nombre       varchar(25) not null,
    Sucursal     varchar(25) not null,
    Sector       varchar(25) not null,
    Cargo        varchar(25) not null,
    Salario      decimal(15,2)
    );'''
exequery(q)

## SUCURSAL

In [10]:
q = "drop table if exists Sucursal;"
exequery(q)
q = """create table if not exists Sucursal (
    IDSucursal         int not null,
    Sucursal           varchar(25),
    Direccion          varchar(50),
    Localidad          varchar(50),
    Provincia          varchar(50),
    Latitud            varchar(100),
    Longitud           varchar(100));"""
exequery(q)

## GASTO

In [11]:
q = "drop table if exists Gasto;"
exequery(q)
q = """create table if not exists Gasto (
   IDGasto      int not null,
   SucursalID   int not null,
   TipoGastoID  int not null,
   Fecha        date,
   Monto        decimal(15,2));"""
exequery(q)

## COMPRA

In [12]:
q = "drop table if exists Compra"
exequery(q)
q = """create table if not exists Compra (
    IDCompra     int not null,
    Fecha        date,
    IdProducto   int not null,
    Cantidad     int not null,
    Precio       decimal(15,2),
    IdProveedor  int not null
    );"""
exequery(q)

## VENTA

In [13]:
q = "drop table if exists Venta;"
exequery(q)
q = """create table if not exists Venta (
    IDVenta        int not null,
    Fecha          date,
    Fecha_Entrega  date,
    IdCanal        int not null,
    IdCliente      int not null,
    IdSucursal     int not null,
    IdEmpleado     int not null,
    IdProducto     int not null,
    Precio         varchar(17) not null,
    Cantidad       varchar(50) not null);"""
exequery(q)

# LLENACIÓN DE TABLAS

## CANAL VENTA

In [14]:
path = "CanalDeVenta.csv"
archive = csvread(path, ",")

for i, row in archive.iterrows():
    exequery("INSERT INTO Canal_Venta values (%s, %s)", tuple(row))

## TIPO GASTO

In [15]:
path = "TiposDeGasto.csv"
ar = csvread(path, ",")

for i, row in ar.iterrows():
    exequery("insert into Tipo_Gasto values (%s, %s, %s)", tuple(row))

## CLIENTE

*Leer CSV*

In [16]:
path = "Clientes.csv"
ar = csvread(path, ";")
ar.info()

for i, row in ar.iterrows():
    cursor.execute("insert into Cliente values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)", tuple(row))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 3407 non-null   int64 
 1   Provincia          3407 non-null   object
 2   Nombre_y_Apellido  3407 non-null   object
 3   Domicilio          3407 non-null   object
 4   Telefono           3407 non-null   object
 5   Edad               3407 non-null   int64 
 6   Localidad          3407 non-null   object
 7   X                  3407 non-null   object
 8   Y                  3407 non-null   object
 9   col10              3407 non-null   object
dtypes: int64(2), object(8)
memory usage: 266.3+ KB


## PROVEEDOR

*Leer CSV*

In [17]:
proveedor = pd.read_csv(r"CSV/Proveedores.csv", index_col=False, delimiter=",")

proveedor.fillna(" ", inplace=True)

proveedor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   IDProveedor  14 non-null     int64 
 1   Nombre       14 non-null     object
 2   Address      14 non-null     object
 3   City         14 non-null     object
 4   State        14 non-null     object
 5   Country      14 non-null     object
 6   departamen   14 non-null     object
dtypes: int64(1), object(6)
memory usage: 912.0+ bytes


*Insertar*

In [18]:
for i, row in proveedor.iterrows():
    q = "insert into Proveedor values (%s,%s,%s,%s,%s,%s,%s);"
    cursor.execute(q, tuple(row))
    conn.commit()

## PRODUCTO

*Leer CSV*

In [19]:
producto = pd.read_csv(r"CSV/PRODUCTOS.csv", index_col=False, delimiter=",")
producto.fillna(" ", inplace=True)
producto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291 entries, 0 to 290
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_PRODUCTO  291 non-null    int64  
 1   Concepto     291 non-null    object 
 2   Tipo         291 non-null    object 
 3    Precio      291 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.2+ KB


*Insertar*

In [20]:
for i, row in producto.iterrows():
    q = "insert into Producto values (%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

## EMPLEADO

*Leer CSV*

In [21]:
empleado = pd.read_csv(r"CSV/Empleados.csv", index_col=False, delimiter=",")
empleado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_empleado  267 non-null    int64  
 1   Apellido     267 non-null    object 
 2   Nombre       267 non-null    object 
 3   Sucursal     267 non-null    object 
 4   Sector       267 non-null    object 
 5   Cargo        267 non-null    object 
 6   Salario      267 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 14.7+ KB


*Insertar*

In [22]:
for i, row in empleado.iterrows():
    q = "insert into Empleado values (%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

## SUCURSAL

*Leer CSV*

In [23]:
sucursal = pd.read_csv(r"CSV/Sucursales.csv", index_col=False, delimiter=";")
sucursal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         31 non-null     int64 
 1   Sucursal   31 non-null     object
 2   Direccion  31 non-null     object
 3   Localidad  31 non-null     object
 4   Provincia  31 non-null     object
 5   Latitud    31 non-null     object
 6   Longitud   31 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


*Insertar*

In [24]:
for i, row in sucursal.iterrows():
    q = "insert into Sucursal values (%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

## GASTO

*Leer CSV*

In [25]:
gasto = pd.read_csv(r"CSV/Gasto.csv", index_col=False, delimiter=",")
gasto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdGasto      8640 non-null   int64  
 1   IdSucursal   8640 non-null   int64  
 2   IdTipoGasto  8640 non-null   int64  
 3   Fecha        8640 non-null   object 
 4   Monto        8640 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 337.6+ KB


*Insertar*

In [26]:
for i, row in gasto.iterrows():
    q = "insert into Gasto values (%s,%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

## COMPRA

*Leer CSV*

In [27]:
compra = pd.read_csv(r"CSV/Compra.csv", index_col=False, delimiter=",")
compra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdCompra     11539 non-null  int64  
 1   Fecha        11539 non-null  object 
 2   IdProducto   11539 non-null  int64  
 3   Cantidad     11539 non-null  int64  
 4   Precio       11539 non-null  float64
 5   IdProveedor  11539 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 541.0+ KB


*Insertar*

In [28]:
for i, row in compra.iterrows():
    q = "insert into Compra values (%s,%s,%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

## VENTA

*Leer CSV*

In [29]:
venta = pd.read_csv(r"CSV/Venta.csv", index_col=False, delimiter=",")
venta.fillna("", inplace=True)
venta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46645 entries, 0 to 46644
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IdVenta        46645 non-null  int64 
 1   Fecha          46645 non-null  object
 2   Fecha_Entrega  46645 non-null  object
 3   IdCanal        46645 non-null  int64 
 4   IdCliente      46645 non-null  int64 
 5   IdSucursal     46645 non-null  int64 
 6   IdEmpleado     46645 non-null  int64 
 7   IdProducto     46645 non-null  int64 
 8   Precio         46645 non-null  object
 9   Cantidad       46645 non-null  object
dtypes: int64(6), object(4)
memory usage: 3.6+ MB


*Insertar*

In [30]:
for i, row in venta.iterrows():
    q = "insert into Venta values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.execute(q, tuple(row))
    conn.commit()

__CERRAR CONEXIÓN__

In [31]:
cursor.close()